In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import cm
import matplotlib.pyplot as plt

# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python3 --version
import metnum

/home/fede2004/workspace/tp3-metnum/notebooks
Python 3.8.2


In [2]:
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.feature_selection import  SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

def errorRMSE(y1,y2 ):
    return sqrt ( mean_squared_error(y1,y2) )

def errorRMSLE(y1,y2 ):
    return sqrt ( mean_squared_log_error(np.abs(y1),np.abs(y2)) )

In [3]:
df = pd.read_csv('../data/train.csv')
#df = df[df['provincia'] == 'Distrito Federal']

df1h = df .copy()

df1h.dropna(subset=['fecha','garages',  'antiguedad'])

#Feature engineering
df1h['es_nuevo']  = df1h['antiguedad'].apply(lambda x: 1 if x <=1 else 0  )

df1h['esta_avenida']  = df1h['direccion'].apply(lambda x: 1 if ("AV." in str(x) or "Av." in str(x) or "av." in str(x) or "Avenida" in str(x) or "avenida" in str(x) or "AVENIDA" in str(x) ) else 0 )

df1h['adicionales'] = df1h['piscina'] +  df1h['gimnasio']  + df1h['usosmultiples'] 
df1h['adicionales']  = df1h['adicionales'].apply(lambda x:   x if x>0 else 0)

df1h['adicionales']  = df1h['adicionales'].apply(lambda x:  (x) )
df1h['adicionales0']  = df1h['adicionales'].apply(lambda x:  (x**0) )
df1h['adicionales2']  = df1h['adicionales'].apply(lambda x:  (x**2) )
df1h['adicionales3']  = df1h['adicionales'].apply(lambda x:  (x**3) )
df1h['adicionales4']  = df1h['adicionales'].apply(lambda x:  (x**4) )

df1h['fecha']  = df1h['fecha'].apply(lambda x:   (int( x.split('-')[0]) + int(x.split('-')[1] )/13.0 )  )
df1h['fecha0']  = df1h['fecha'].apply(lambda x:   x**0 )
df1h['fecha2']  = df1h['fecha'].apply(lambda x:  x **2  )
df1h['fecha3']  = df1h['fecha'].apply(lambda x:   x **3  )
df1h['fecha4']  = df1h['fecha'].apply(lambda x:  x **4 )

df1h['descripcion']  = df1h['descripcion'].apply(lambda x:   ( len ( str ( x))) )
pd.set_option('display.max_columns', None)

df1h['cercanias'] = df1h['centroscomercialescercanos'] +  df1h['escuelascercanas']   
df1h['cercanias']  = df1h['cercanias'].apply(lambda x:  (x) )
df1h['cercanias0']  = df1h['cercanias'].apply(lambda x:  (x**0) )
df1h['cercanias2']  = df1h['cercanias'].apply(lambda x:  (x**2) )
df1h['cercanias3']  = df1h['cercanias'].apply(lambda x:  (x**3) )
df1h['cercanias4']  = df1h['cercanias'].apply(lambda x:  (x**4) )

#Hot encoding
df1h = pd.concat([df1h,pd.get_dummies(df1h['tipodepropiedad'], prefix='tipodepropiedad_', dtype=float)],axis=1)
# df1h = pd.concat([df1h,pd.get_dummies(df1h['ciudad'], prefix='ciudad_', dtype=float)],axis=1)




#polinomios de var númericas.
df1h['garages']  = df1h['garages'].apply(lambda x:  (x) )
df1h['garages0']  = df1h['garages'].apply(lambda x:  (x**0) )
df1h['garages2']  = df1h['garages'].apply(lambda x:  (x**2) )
df1h['garages3']  = df1h['garages'].apply(lambda x:  (x**3) )
df1h['garages4']  = df1h['garages'].apply(lambda x:  (x**4) )


df1h['banos0']  = df1h['banos'].apply(lambda x:  (x**0) )
df1h['banos1']  = df1h['banos'].apply(lambda x:  (x**1) )
df1h['banos2']  = df1h['banos'].apply(lambda x:  (x**2) )
df1h['banos3']  = df1h['banos'].apply(lambda x:  (x**3 ))
df1h['banos4']  = df1h['banos'].apply(lambda x:  (x**4) )

df1h['metroscubiertos']  = df1h['metroscubiertos'].apply(lambda x:  (x) )
df1h['metroscubiertos0']  = df1h['metroscubiertos'].apply(lambda x:  (x**0) )
df1h['metroscubiertos2']  = df1h['metroscubiertos'].apply(lambda x:  (x**2) )
df1h['metroscubiertos3']  = df1h['metroscubiertos'].apply(lambda x:  (x**3) )
df1h['metroscubiertos4']  = df1h['metroscubiertos'].apply(lambda x:  (x**4) )


df1h['habitaciones']  = df1h['habitaciones'].apply(lambda x:  (x) )
df1h['habitaciones0']  = df1h['habitaciones'].apply(lambda x:  (x**0) )
df1h['habitaciones2']  = df1h['habitaciones'].apply(lambda x:  (x**2) )
df1h['habitaciones3']  = df1h['habitaciones'].apply(lambda x:  (x**3) )
df1h['habitaciones4']  = df1h['habitaciones'].apply(lambda x:  (x**4) )


df1h['metrostotales']  = df1h['metrostotales'].apply(lambda x:  (x) )
df1h['metrostotales0']  = df1h['metrostotales'].apply(lambda x:  (x**0) )
df1h['metrostotales2']  = df1h['metrostotales'].apply(lambda x:  (x**2) )
df1h['metrostotales3']  = df1h['metrostotales'].apply(lambda x:  (x**3) )
df1h['metrostotales4']  = df1h['metrostotales'].apply(lambda x:  (x**4) )


df1h.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,es_nuevo,esta_avenida,adicionales,adicionales0,adicionales2,adicionales3,adicionales4,fecha0,fecha2,fecha3,fecha4,cercanias,cercanias0,cercanias2,cercanias3,cercanias4,tipodepropiedad__Apartamento,tipodepropiedad__Bodega comercial,tipodepropiedad__Casa,tipodepropiedad__Casa en condominio,tipodepropiedad__Casa uso de suelo,tipodepropiedad__Departamento Compartido,tipodepropiedad__Duplex,tipodepropiedad__Edificio,tipodepropiedad__Garage,tipodepropiedad__Hospedaje,tipodepropiedad__Huerta,tipodepropiedad__Inmuebles productivos urbanos,tipodepropiedad__Local Comercial,tipodepropiedad__Local en centro comercial,tipodepropiedad__Lote,tipodepropiedad__Nave industrial,tipodepropiedad__Oficina comercial,tipodepropiedad__Otros,tipodepropiedad__Quinta Vacacional,tipodepropiedad__Rancho,tipodepropiedad__Terreno,tipodepropiedad__Terreno comercial,tipodepropiedad__Terreno industrial,tipodepropiedad__Villa,garages0,garages2,garages3,garages4,banos0,banos1,banos2,banos3,banos4,metroscubiertos0,metroscubiertos2,metroscubiertos3,metroscubiertos4,habitaciones0,habitaciones2,habitaciones3,habitaciones4,metrostotales0,metrostotales2,metrostotales3,metrostotales4
0,254099,depto. tipo a-402,171,Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,NaN,NaN,2015.615385,0.0,0.0,0.0,0.0,0.0,2273000.0,0,1,0.0,1.0,0.0,0.0,0.0,1.0,4.062705e+06,8.188851e+09,1.650557e+13,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,8.0,16.0,1.0,6400.0,512000.0,4.096000e+07,1.0,4.0,8.0,16.0,1.0,6400.0,512000.0,4.096000e+07
1,53461,condominio horizontal en venta,716,Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,19.310205,-99.227655,2013.461538,0.0,0.0,0.0,1.0,1.0,3600000.0,0,1,0.0,1.0,0.0,0.0,0.0,1.0,4.054027e+06,8.162628e+09,1.643514e+13,2.0,1.0,4.0,8.0,16.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,8.0,16.0,1.0,2.0,4.0,8.0,16.0,1.0,71824.0,19248832.0,5.158687e+09,1.0,9.0,27.0,81.0,1.0,32400.0,5832000.0,1.049760e+09
2,247984,casa en venta urbi 3 recamaras tonala,1340,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,2.0,144.0,166.0,48551.0,NaN,NaN,2015.769231,0.0,0.0,0.0,0.0,0.0,1200000.0,0,0,0.0,1.0,0.0,0.0,0.0,1.0,4.063326e+06,8.190727e+09,1.651061e+13,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,8.0,16.0,1.0,2.0,4.0,8.0,16.0,1.0,20736.0,2985984.0,4.299817e+08,1.0,9.0,27.0,81.0,1.0,27556.0,4574296.0,7.593331e+08
3,209067,casa sola en toluca zinacantepec con credito i...,357,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,1.0,63.0,67.0,53666.0,19.301890,-99.688015,2012.230769,0.0,0.0,0.0,1.0,1.0,650000.0,1,0,0.0,1.0,0.0,0.0,0.0,1.0,4.049073e+06,8.147669e+09,1.639499e+13,2.0,1.0,4.0,8.0,16.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3969.0,250047.0,1.575296e+07,1.0,4.0,8.0,16.0,1.0,4489.0,300763.0,2.015112e+07
4,185997,paseos del sol,103,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,1.0,95.0,95.0,47835.0,NaN,NaN,2016.461538,0.0,0.0,0.0,0.0,0.0,1150000.0,0,0,0.0,1.0,0.0,0.0,0.0,1.0,4.066117e+06,8.199169e+09,1.653331e+13,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,9025.0,857375.0,8.145062e+07,1.0,4.0,8.0,16.0,1.0,9025.0,857375.0,8.145062e+07


In [4]:
df1h = df1h.loc[:, df1h.columns != 'titulo']
# df1h = df1h.loc[:, df1h.columns != 'metroscubiertos']
# df1h = df1h.loc[:, df1h.columns != 'descripcion']
df1h = df1h.loc[:, df1h.columns != 'direccion']
df1h = df1h.loc[:, df1h.columns != 'provincia']
df1h = df1h.loc[:, df1h.columns != 'ciudad']
df1h = df1h.loc[:, df1h.columns != 'id']
# df1h = df1h.loc[:, df1h.columns != 'fecha']
df1h = df1h.loc[:, df1h.columns != 'lat']
df1h = df1h.loc[:, df1h.columns != 'lng']
df1h = df1h.loc[:, df1h.columns != 'idzona']
df1h = df1h.loc[:, df1h.columns != 'tipodepropiedad']
df1h = df1h.loc[:, df1h.columns != 'antiguedad']
# df1h = df1h.loc[:, df1h.columns != 'banos']
# df1h = df1h.loc[:, df1h.columns != 'garages']
df1h = df1h.loc[:, df1h.columns != 'usosmultiples']
df1h = df1h.loc[:, df1h.columns != 'gimnasio']
df1h = df1h.loc[:, df1h.columns != 'piscina']
df1h = df1h.loc[:, df1h.columns != 'escuelascercanas']
df1h = df1h.loc[:, df1h.columns != 'centroscomercialescercanos']
# df1h = df1h.loc[:, df1h.columns != 'habitaciones']
# df1h = df1h.loc[:, df1h.columns != 'metrostotales']



df1h = df1h.dropna( )
# df1h = df1h[df1h['habitaciones']>0]
df1h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139408 entries, 0 to 239998
Data columns (total 69 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   descripcion                                     139408 non-null  int64  
 1   habitaciones                                    139408 non-null  float64
 2   garages                                         139408 non-null  float64
 3   banos                                           139408 non-null  float64
 4   metroscubiertos                                 139408 non-null  float64
 5   metrostotales                                   139408 non-null  float64
 6   fecha                                           139408 non-null  float64
 7   precio                                          139408 non-null  float64
 8   es_nuevo                                        139408 non-null  int64  
 9   esta_avenida              

In [5]:
dfsp = df1h
dfsp = dfsp.loc[:, df1h.columns != 'precio']

X = dfsp.values
n =len(dfsp.columns)

y = df1h['precio'].values
X = X.reshape(len(X), n)
y = y.reshape(len(y),)

from sklearn.model_selection import KFold

accuraccy = 0
splits = 5
error_rmse = 0 
error_rmsle = 0

kf = KFold(n_splits=splits, random_state=42, shuffle=True)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lr = metnum.LinearRegression()

    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)

    error_rmse += errorRMSE(y_test,y_pred)
    error_rmsle += errorRMSLE(y_test,y_pred)
    
    accuraccy += errorRMSE(y_test,y_pred) / y.mean()
    
print(error_rmse/splits)
print(error_rmsle/splits)
print(accuraccy/splits)

1127590.24448397
0.6758680970615456
0.5260959946357672


In [6]:
# Ejecucion básica
#0.6310344040291046

# Agrego los polinomios
# 0.6043233599338175

# Agrego la fecha
# 0.5908512108428354

# Agrego si es nuevo
# 0.590771895418362

# Agrego avenida
# 0.5900300275506789

# Agrego adicionales sin polinomio
# 0.5804374055790497

# Agrego adicionales con polinomio
# 0.5785602231361093

# Agrego descripción
# 0.5783969608451345

# Agrego cercanias con polinomio
# 0.5776072697013944

# Agrego hot encoding tipo propiedad
# 0.5260959946357672

# Agrego hot encoding ciudad



0.43517305049327226


0.43517305049327226